# Train SALLY ensemble

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma_sys/'

## Settings

In [5]:
n_estimators = 10

## Training function

In [10]:
def train_ensemble(filename, use_tight_cuts=True, use_antitight_cuts=False, n_estimators=n_estimators, **kwargs):
    cut_label = '_tight' if use_tight_cuts else '_antitight' if use_antitight_cuts else ''
    
    ensemble = EnsembleForge(n_estimators)

    ensemble.train_all(
        method='sally',
        x_filename=[sample_dir + 'train_local{}/x_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        t_xz0_filename=[sample_dir + 'train_local{}/t_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        **kwargs
    )

    ensemble.save(model_dir + 'sally_ensemble_' + filename)

## All observables

In [11]:
train_ensemble(
    'all',
    use_tight_cuts=False,
    validation_split=0.5,
    early_stopping=True
)

14:14 madminer.ml          INFO    Training 10 estimators in ensemble
14:14 madminer.ml          INFO    Training estimator 1 / 10 in ensemble
14:14 madminer.ml          INFO    Starting training
14:14 madminer.ml          INFO      Method:                 sally
14:14 madminer.ml          INFO      Training data:          x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/x_train_0.npy
14:14 madminer.ml          INFO                              t_xz (theta0) at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/t_xz_train_0.npy
14:14 madminer.ml          INFO      Features:               all
14:14 madminer.ml          INFO      Method:                 sally
14:14 madminer.ml          INFO      Hidden layers:          (100, 100)
14:14 madminer.ml          INFO      Activation function:    tanh
14:14 madminer.ml          INFO      Batch size:             128
14:14 madminer.ml          INFO      

16:06 madminer.ml          INFO      Activation function:    tanh
16:06 madminer.ml          INFO      Batch size:             128
16:06 madminer.ml          INFO      Trainer:                amsgrad
16:06 madminer.ml          INFO      Epochs:                 50
16:06 madminer.ml          INFO      Learning rate:          0.001 initially, decaying to 0.0001
16:06 madminer.ml          INFO      Validation split:       0.5
16:06 madminer.ml          INFO      Early stopping:         True
16:06 madminer.ml          INFO      Scale inputs:           True
16:06 madminer.ml          INFO      Shuffle labels          False
16:06 madminer.ml          INFO      Regularization:         None
16:06 madminer.ml          INFO      Samples:                all
16:06 madminer.ml          INFO    Loading training data
16:06 madminer.ml          INFO    Found 1000000 samples with 34 parameters and 33 observables
16:06 madminer.ml          INFO    Rescaling inputs
16:06 madminer.ml          INFO    Creat

16:52 madminer.ml          INFO    Found 1000000 samples with 34 parameters and 33 observables
16:52 madminer.ml          INFO    Rescaling inputs
16:52 madminer.ml          INFO    Creating model for method sally
16:52 madminer.ml          INFO    Training model
16:54 madminer.utils.ml.sc INFO      Epoch 05: train loss 0.1060 (mse_score: 0.1060)
16:54 madminer.utils.ml.sc INFO                val. loss  0.1430 (mse_score: 0.1430) (*)
16:57 madminer.utils.ml.sc INFO      Epoch 10: train loss 0.0858 (mse_score: 0.0858)
16:57 madminer.utils.ml.sc INFO                val. loss  0.1323 (mse_score: 0.1323) (*)
16:59 madminer.utils.ml.sc INFO      Epoch 15: train loss 0.0763 (mse_score: 0.0763)
16:59 madminer.utils.ml.sc INFO                val. loss  0.1258 (mse_score: 0.1258) (*)
17:01 madminer.utils.ml.sc INFO      Epoch 20: train loss 0.0699 (mse_score: 0.0699)
17:01 madminer.utils.ml.sc INFO                val. loss  0.1260 (mse_score: 0.1260)
17:03 madminer.utils.ml.sc INFO      Epoch 2

17:57 madminer.utils.ml.sc INFO      Epoch 20: train loss 0.0546 (mse_score: 0.0546)
17:57 madminer.utils.ml.sc INFO                val. loss  0.1867 (mse_score: 0.1867)
18:00 madminer.utils.ml.sc INFO      Epoch 25: train loss 0.0498 (mse_score: 0.0498)
18:00 madminer.utils.ml.sc INFO                val. loss  0.1848 (mse_score: 0.1848) (*)
18:02 madminer.utils.ml.sc INFO      Epoch 30: train loss 0.0464 (mse_score: 0.0464)
18:02 madminer.utils.ml.sc INFO                val. loss  0.1837 (mse_score: 0.1837) (*)
18:05 madminer.utils.ml.sc INFO      Epoch 35: train loss 0.0437 (mse_score: 0.0437)
18:05 madminer.utils.ml.sc INFO                val. loss  0.1879 (mse_score: 0.1879)
18:08 madminer.utils.ml.sc INFO      Epoch 40: train loss 0.0418 (mse_score: 0.0418)
18:08 madminer.utils.ml.sc INFO                val. loss  0.1846 (mse_score: 0.1846)
18:10 madminer.utils.ml.sc INFO      Epoch 45: train loss 0.0404 (mse_score: 0.0404)
18:10 madminer.utils.ml.sc INFO                val. loss 

19:02 madminer.utils.ml.sc INFO      Epoch 45: train loss 0.0828 (mse_score: 0.0828)
19:02 madminer.utils.ml.sc INFO                val. loss  0.1900 (mse_score: 0.1900)
19:06 madminer.utils.ml.sc INFO      Epoch 50: train loss 0.0815 (mse_score: 0.0815)
19:06 madminer.utils.ml.sc INFO                val. loss  0.1905 (mse_score: 0.1905)
19:06 madminer.utils.ml.sc INFO    Early stopping after epoch 42, with loss 0.19 compared to final loss 0.19
19:06 madminer.utils.ml.sc INFO    Finished training
19:06 madminer.ml          INFO    Training estimator 10 / 10 in ensemble
19:06 madminer.ml          INFO    Starting training
19:06 madminer.ml          INFO      Method:                 sally
19:06 madminer.ml          INFO      Training data:          x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/x_train_9.npy
19:06 madminer.ml          INFO                              t_xz (theta0) at /Users/johannbrehmer/work/projects/madminer/diboson

In [ ]:
train_ensemble(
    'all_tight',
    use_tight_cuts=True,
    validation_split=0.5,
    early_stopping=True
)

In [15]:
train_ensemble(
    'all_antitight',
    use_tight_cuts=False,
    use_antitight_cuts=True,
    validation_split=0.5,
    early_stopping=True
)

17:58 madminer.ml          INFO    Training 10 estimators in ensemble
17:58 madminer.ml          INFO    Training estimator 1 / 10 in ensemble
17:58 madminer.ml          INFO    Starting training
17:58 madminer.ml          INFO      Method:                 sally
17:58 madminer.ml          INFO      Training data:          x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local_antitight/x_train_0.npy
17:58 madminer.ml          INFO                              t_xz (theta0) at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local_antitight/t_xz_train_0.npy
17:58 madminer.ml          INFO      Features:               all
17:58 madminer.ml          INFO      Method:                 sally
17:58 madminer.ml          INFO      Hidden layers:          (100, 100)
17:58 madminer.ml          INFO      Activation function:    tanh
17:58 madminer.ml          INFO      Batch size:             128
17:58 madminer.ml

18:12 madminer.utils.ml.sc INFO                val. loss  0.0327 (mse_score: 0.0327)
18:12 madminer.utils.ml.sc INFO      Epoch 38: train loss 0.0371 (mse_score: 0.0371)
18:12 madminer.utils.ml.sc INFO                val. loss  0.0324 (mse_score: 0.0324) (*)
18:12 madminer.utils.ml.sc INFO      Epoch 39: train loss 0.0370 (mse_score: 0.0370)
18:12 madminer.utils.ml.sc INFO                val. loss  0.0324 (mse_score: 0.0324)
18:13 madminer.utils.ml.sc INFO      Epoch 40: train loss 0.0368 (mse_score: 0.0368)
18:13 madminer.utils.ml.sc INFO                val. loss  0.0325 (mse_score: 0.0325)
18:13 madminer.utils.ml.sc INFO      Epoch 41: train loss 0.0367 (mse_score: 0.0367)
18:13 madminer.utils.ml.sc INFO                val. loss  0.0322 (mse_score: 0.0322) (*)
18:14 madminer.utils.ml.sc INFO      Epoch 42: train loss 0.0366 (mse_score: 0.0366)
18:14 madminer.utils.ml.sc INFO                val. loss  0.0324 (mse_score: 0.0324)
18:14 madminer.utils.ml.sc INFO      Epoch 43: train loss

18:26 madminer.utils.ml.sc INFO                val. loss  0.0248 (mse_score: 0.0248)
18:26 madminer.utils.ml.sc INFO      Epoch 24: train loss 0.0236 (mse_score: 0.0236)
18:26 madminer.utils.ml.sc INFO                val. loss  0.0236 (mse_score: 0.0236) (*)
18:26 madminer.utils.ml.sc INFO      Epoch 25: train loss 0.0233 (mse_score: 0.0233)
18:26 madminer.utils.ml.sc INFO                val. loss  0.0231 (mse_score: 0.0231) (*)
18:27 madminer.utils.ml.sc INFO      Epoch 26: train loss 0.0231 (mse_score: 0.0231)
18:27 madminer.utils.ml.sc INFO                val. loss  0.0238 (mse_score: 0.0238)
18:27 madminer.utils.ml.sc INFO      Epoch 27: train loss 0.0229 (mse_score: 0.0229)
18:27 madminer.utils.ml.sc INFO                val. loss  0.0230 (mse_score: 0.0230) (*)
18:28 madminer.utils.ml.sc INFO      Epoch 28: train loss 0.0227 (mse_score: 0.0227)
18:28 madminer.utils.ml.sc INFO                val. loss  0.0230 (mse_score: 0.0230) (*)
18:28 madminer.utils.ml.sc INFO      Epoch 29: tr

18:42 madminer.utils.ml.sc INFO                val. loss  0.0592 (mse_score: 0.0592) (*)
18:42 madminer.utils.ml.sc INFO      Epoch 10: train loss 0.0394 (mse_score: 0.0394)
18:42 madminer.utils.ml.sc INFO                val. loss  0.0598 (mse_score: 0.0598)
18:43 madminer.utils.ml.sc INFO      Epoch 11: train loss 0.0384 (mse_score: 0.0384)
18:43 madminer.utils.ml.sc INFO                val. loss  0.0584 (mse_score: 0.0584) (*)
18:43 madminer.utils.ml.sc INFO      Epoch 12: train loss 0.0374 (mse_score: 0.0374)
18:43 madminer.utils.ml.sc INFO                val. loss  0.0585 (mse_score: 0.0585)
18:44 madminer.utils.ml.sc INFO      Epoch 13: train loss 0.0370 (mse_score: 0.0370)
18:44 madminer.utils.ml.sc INFO                val. loss  0.0575 (mse_score: 0.0575) (*)
18:44 madminer.utils.ml.sc INFO      Epoch 14: train loss 0.0362 (mse_score: 0.0362)
18:44 madminer.utils.ml.sc INFO                val. loss  0.0572 (mse_score: 0.0572) (*)
18:45 madminer.utils.ml.sc INFO      Epoch 15: tr

19:01 madminer.ml          INFO      Epochs:                 50
19:01 madminer.ml          INFO      Learning rate:          0.001 initially, decaying to 0.0001
19:01 madminer.ml          INFO      Validation split:       0.5
19:01 madminer.ml          INFO      Early stopping:         True
19:01 madminer.ml          INFO      Scale inputs:           True
19:01 madminer.ml          INFO      Shuffle labels          False
19:01 madminer.ml          INFO      Regularization:         None
19:01 madminer.ml          INFO      Samples:                all
19:01 madminer.ml          INFO    Loading training data
19:01 madminer.ml          INFO    Found 1000000 samples with 34 parameters and 33 observables
19:01 madminer.ml          INFO    Rescaling inputs
19:01 madminer.ml          INFO    Creating model for method sally
19:01 madminer.ml          INFO    Training model
19:02 madminer.utils.ml.sc INFO      Epoch 01: train loss 0.0977 (mse_score: 0.0977)
19:02 madminer.utils.ml.sc INFO       

...

20:04 madminer.utils.ml.sc INFO                val. loss  0.0603 (mse_score: 0.0603) (*)
20:04 madminer.utils.ml.sc INFO      Epoch 02: train loss 0.0561 (mse_score: 0.0561)
20:04 madminer.utils.ml.sc INFO                val. loss  0.0477 (mse_score: 0.0477) (*)
20:05 madminer.utils.ml.sc INFO      Epoch 03: train loss 0.0448 (mse_score: 0.0448)
20:05 madminer.utils.ml.sc INFO                val. loss  0.0405 (mse_score: 0.0405) (*)
20:05 madminer.utils.ml.sc INFO      Epoch 04: train loss 0.0393 (mse_score: 0.0393)
20:05 madminer.utils.ml.sc INFO                val. loss  0.0364 (mse_score: 0.0364) (*)
20:05 madminer.utils.ml.sc INFO      Epoch 05: train loss 0.0355 (mse_score: 0.0355)
20:05 madminer.utils.ml.sc INFO                val. loss  0.0344 (mse_score: 0.0344) (*)
20:06 madminer.utils.ml.sc INFO      Epoch 06: train loss 0.0330 (mse_score: 0.0330)
20:06 madminer.utils.ml.sc INFO                val. loss  0.0357 (mse_score: 0.0357)
20:06 madminer.utils.ml.sc INFO      Epoch 07

20:24 madminer.utils.ml.sc INFO      Epoch 49: train loss 0.0160 (mse_score: 0.0160)
20:24 madminer.utils.ml.sc INFO                val. loss  0.0245 (mse_score: 0.0245) (*)
20:24 madminer.utils.ml.sc INFO      Epoch 50: train loss 0.0160 (mse_score: 0.0160)
20:24 madminer.utils.ml.sc INFO                val. loss  0.0246 (mse_score: 0.0246)
20:24 madminer.utils.ml.sc INFO    Early stopping after epoch 49, with loss 0.02 compared to final loss 0.02
20:24 madminer.utils.ml.sc INFO    Finished training
20:24 madminer.ml          INFO    Training estimator 8 / 10 in ensemble
20:24 madminer.ml          INFO    Starting training
20:24 madminer.ml          INFO      Method:                 sally
20:24 madminer.ml          INFO      Training data:          x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local_antitight/x_train_7.npy
20:24 madminer.ml          INFO                              t_xz (theta0) at /Users/johannbrehmer/work/projects/mad

02:09 madminer.utils.ml.sc INFO      Epoch 35: train loss 0.0294 (mse_score: 0.0294)
02:09 madminer.utils.ml.sc INFO                val. loss  0.0223 (mse_score: 0.0223) (*)
02:10 madminer.utils.ml.sc INFO      Epoch 36: train loss 0.0292 (mse_score: 0.0292)
02:10 madminer.utils.ml.sc INFO                val. loss  0.0224 (mse_score: 0.0224)
02:10 madminer.utils.ml.sc INFO      Epoch 37: train loss 0.0290 (mse_score: 0.0290)
02:10 madminer.utils.ml.sc INFO                val. loss  0.0228 (mse_score: 0.0228)
02:10 madminer.utils.ml.sc INFO      Epoch 38: train loss 0.0288 (mse_score: 0.0288)
02:10 madminer.utils.ml.sc INFO                val. loss  0.0224 (mse_score: 0.0224)
02:11 madminer.utils.ml.sc INFO      Epoch 39: train loss 0.0287 (mse_score: 0.0287)
02:11 madminer.utils.ml.sc INFO                val. loss  0.0222 (mse_score: 0.0222) (*)
02:11 madminer.utils.ml.sc INFO      Epoch 40: train loss 0.0285 (mse_score: 0.0285)
02:11 madminer.utils.ml.sc INFO                val. loss 

02:22 madminer.utils.ml.sc INFO      Epoch 21: train loss 0.0193 (mse_score: 0.0193)
02:22 madminer.utils.ml.sc INFO                val. loss  0.0333 (mse_score: 0.0333) (*)
02:23 madminer.utils.ml.sc INFO      Epoch 22: train loss 0.0190 (mse_score: 0.0190)
02:23 madminer.utils.ml.sc INFO                val. loss  0.0330 (mse_score: 0.0330) (*)
02:23 madminer.utils.ml.sc INFO      Epoch 23: train loss 0.0187 (mse_score: 0.0187)
02:23 madminer.utils.ml.sc INFO                val. loss  0.0341 (mse_score: 0.0341)
02:23 madminer.utils.ml.sc INFO      Epoch 24: train loss 0.0185 (mse_score: 0.0185)
02:23 madminer.utils.ml.sc INFO                val. loss  0.0331 (mse_score: 0.0331)
02:24 madminer.utils.ml.sc INFO      Epoch 25: train loss 0.0184 (mse_score: 0.0184)
02:24 madminer.utils.ml.sc INFO                val. loss  0.0328 (mse_score: 0.0328) (*)
02:24 madminer.utils.ml.sc INFO      Epoch 26: train loss 0.0181 (mse_score: 0.0181)
02:24 madminer.utils.ml.sc INFO                val. l

02:36 madminer.utils.ml.sc INFO      Epoch 07: train loss 0.0483 (mse_score: 0.0483)
02:36 madminer.utils.ml.sc INFO                val. loss  0.0370 (mse_score: 0.0370) (*)
02:37 madminer.utils.ml.sc INFO      Epoch 08: train loss 0.0464 (mse_score: 0.0464)
02:37 madminer.utils.ml.sc INFO                val. loss  0.0364 (mse_score: 0.0364) (*)
02:37 madminer.utils.ml.sc INFO      Epoch 09: train loss 0.0450 (mse_score: 0.0450)
02:37 madminer.utils.ml.sc INFO                val. loss  0.0360 (mse_score: 0.0360) (*)
02:37 madminer.utils.ml.sc INFO      Epoch 10: train loss 0.0439 (mse_score: 0.0439)
02:37 madminer.utils.ml.sc INFO                val. loss  0.0350 (mse_score: 0.0350) (*)
02:38 madminer.utils.ml.sc INFO      Epoch 11: train loss 0.0428 (mse_score: 0.0428)
02:38 madminer.utils.ml.sc INFO                val. loss  0.0343 (mse_score: 0.0343) (*)
02:38 madminer.utils.ml.sc INFO      Epoch 12: train loss 0.0420 (mse_score: 0.0420)
02:38 madminer.utils.ml.sc INFO              

## Minimal observable basis (no jets)

In [ ]:
min_obs = [0,1] + list(range(4,12)) + list(range(16,33))

In [14]:
train_ensemble(
    'minimal',
    use_tight_cuts=False,
    features=[min_obs for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True
)

21:52 madminer.ml          INFO    Training 10 estimators in ensemble
21:52 madminer.ml          INFO    Training estimator 1 / 10 in ensemble
21:52 madminer.ml          INFO    Starting training
21:52 madminer.ml          INFO      Method:                 sally
21:52 madminer.ml          INFO      Training data:          x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/x_train_0.npy
21:52 madminer.ml          INFO                              t_xz (theta0) at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/t_xz_train_0.npy
21:52 madminer.ml          INFO      Features:               [0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
21:52 madminer.ml          INFO      Method:                 sally
21:52 madminer.ml          INFO      Hidden layers:          (100, 100)
21:52 madminer.ml          INFO      Activation function:    tanh
21:52

07:04 madminer.ml          INFO                              t_xz (theta0) at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/t_xz_train_2.npy
07:04 madminer.ml          INFO      Features:               [0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
07:04 madminer.ml          INFO      Method:                 sally
07:04 madminer.ml          INFO      Hidden layers:          (100, 100)
07:04 madminer.ml          INFO      Activation function:    tanh
07:04 madminer.ml          INFO      Batch size:             128
07:04 madminer.ml          INFO      Trainer:                amsgrad
07:04 madminer.ml          INFO      Epochs:                 50
07:04 madminer.ml          INFO      Learning rate:          0.001 initially, decaying to 0.0001
07:04 madminer.ml          INFO      Validation split:       0.5
07:04 madminer.ml          INFO      Early stopping:         True
07:04 madminer.ml

11:16 madminer.ml          INFO      Activation function:    tanh
11:16 madminer.ml          INFO      Batch size:             128
11:16 madminer.ml          INFO      Trainer:                amsgrad
11:16 madminer.ml          INFO      Epochs:                 50
11:16 madminer.ml          INFO      Learning rate:          0.001 initially, decaying to 0.0001
11:16 madminer.ml          INFO      Validation split:       0.5
11:16 madminer.ml          INFO      Early stopping:         True
11:16 madminer.ml          INFO      Scale inputs:           True
11:16 madminer.ml          INFO      Shuffle labels          False
11:16 madminer.ml          INFO      Regularization:         None
11:16 madminer.ml          INFO      Samples:                all
11:16 madminer.ml          INFO    Loading training data
11:16 madminer.ml          INFO    Found 1000000 samples with 34 parameters and 33 observables
11:16 madminer.ml          INFO    Rescaling inputs
11:16 madminer.ml          INFO    Only 

12:11 madminer.ml          INFO      Early stopping:         True
12:11 madminer.ml          INFO      Scale inputs:           True
12:11 madminer.ml          INFO      Shuffle labels          False
12:11 madminer.ml          INFO      Regularization:         None
12:11 madminer.ml          INFO      Samples:                all
12:11 madminer.ml          INFO    Loading training data
12:11 madminer.ml          INFO    Found 1000000 samples with 34 parameters and 33 observables
12:11 madminer.ml          INFO    Rescaling inputs
12:11 madminer.ml          INFO    Only using 27 of 33 observables
12:11 madminer.ml          INFO    Creating model for method sally
12:11 madminer.ml          INFO    Training model
12:14 madminer.utils.ml.sc INFO      Epoch 05: train loss 0.0887 (mse_score: 0.0887)
12:14 madminer.utils.ml.sc INFO                val. loss  0.2115 (mse_score: 0.2115) (*)
12:16 madminer.utils.ml.sc INFO      Epoch 10: train loss 0.0706 (mse_score: 0.0706)
12:16 madminer.utils.ml

13:06 madminer.ml          INFO    Rescaling inputs
13:06 madminer.ml          INFO    Only using 27 of 33 observables
13:06 madminer.ml          INFO    Creating model for method sally
13:06 madminer.ml          INFO    Training model
13:09 madminer.utils.ml.sc INFO      Epoch 05: train loss 0.2005 (mse_score: 0.2005)
13:09 madminer.utils.ml.sc INFO                val. loss  0.1568 (mse_score: 0.1568) (*)
13:12 madminer.utils.ml.sc INFO      Epoch 10: train loss 0.1790 (mse_score: 0.1790)
13:12 madminer.utils.ml.sc INFO                val. loss  0.1456 (mse_score: 0.1456) (*)
13:15 madminer.utils.ml.sc INFO      Epoch 15: train loss 0.1658 (mse_score: 0.1658)
13:15 madminer.utils.ml.sc INFO                val. loss  0.1389 (mse_score: 0.1389) (*)
13:18 madminer.utils.ml.sc INFO      Epoch 20: train loss 0.1580 (mse_score: 0.1580)
13:18 madminer.utils.ml.sc INFO                val. loss  0.1367 (mse_score: 0.1367) (*)
13:21 madminer.utils.ml.sc INFO      Epoch 25: train loss 0.1525 (ms

In [ ]:
train_ensemble(
    'minimal_tight',
    use_tight_cuts=True,
    features=[min_obs for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True
)

## Just resurrection phi

In [ ]:
train_ensemble(
    'phi_tight',
    use_tight_cuts=True,
    features=[[32] for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True,
)

In [ ]:
train_ensemble(
    'pta_phi_tight',
    use_tight_cuts=True,
    features=[[9, 32] for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True,
)